# 🚀 Hierarchical Subchat System - Kaggle GPU Testing

## 📋 Setup Checklist (Do Once):

### 1️⃣ **Add Kaggle Secrets** (Most Important!)
Go to: **https://www.kaggle.com/settings** → Add-ons → Secrets

Add these two secrets:
- **`GROQ_API_KEY`** = Your Groq API key (for query decomposition)
- **`GITHUB_TOKEN`** = Your GitHub personal access token (for pushing results)

### 2️⃣ **Enable Internet in This Notebook**
- Click "⚙️ Settings" (top right)
- Turn ON **"Internet"** toggle
- Click "Save"

### 3️⃣ **Make Sure This Notebook is PRIVATE**
- Never share secrets in public notebooks!

---

## ▶️ Run Order:
1. **Cells 2-7**: Load secrets and libraries
2. **Cell 8**: Set environment variables (LLM_BACKEND=vllm)
3. **Cell 9**: Load vLLM model on Kaggle GPUs (Qwen-3 14B AWQ) - **Takes 2-3 minutes**
4. **Cells 15-17**: Clone repo, configure git, run test log push
5. **Cell 21**: Integrate vLLM with backend (registers model globally)
6. **Cell 23**: Test vLLM integration
7. **Run your actual tests**: Execute test scripts in backend/dataset/
8. **Push results**: Use git_commit_and_push() function to sync to GitHub

---

## 🎯 What This Notebook Does:
- ✅ Loads vLLM model (Qwen-3 14B) on Kaggle's 2x GPUs with AWQ quantization
- ✅ Integrates vLLM with your backend via `VLLMClient` singleton
- ✅ Runs hierarchical subchat tests using GPU-accelerated inference
- ✅ Generates performance logs for buffer size analysis
- ✅ Automatically syncs results back to your GitHub repo

---

## 🔧 Technical Details:
- **vLLM Config**: Tensor parallelism across both GPUs, 91% memory utilization
- **Backend Mode**: `LLM_BACKEND=vllm` (set in cell 8)
- **Model**: Qwen-3 14B AWQ (5120 max tokens, prefix caching enabled)
- **No .env needed**: Secrets loaded from Kaggle environment
- **Git Workflow**: Clone → Test → Push results to `kaggle-run` branch

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/qwen2.5/transformers/1.5b-instruct-awq/1/config.json
/kaggle/input/qwen2.5/transformers/1.5b-instruct-awq/1/merges.txt
/kaggle/input/qwen2.5/transformers/1.5b-instruct-awq/1/LICENSE
/kaggle/input/qwen2.5/transformers/1.5b-instruct-awq/1/README.md
/kaggle/input/qwen2.5/transformers/1.5b-instruct-awq/1/tokenizer.json
/kaggle/input/qwen2.5/transformers/1.5b-instruct-awq/1/vocab.json
/kaggle/input/qwen2.5/transformers/1.5b-instruct-awq/1/tokenizer_config.json
/kaggle/input/qwen2.5/transformers/1.5b-instruct-awq/1/model.safetensors
/kaggle/input/qwen2.5/transformers/1.5b-instruct-awq/1/.gitattributes
/kaggle/input/qwen2.5/transformers/1.5b-instruct-awq/1/generation_config.json
/kaggle/input/qwen2.5/transformers/14b-instruct-awq/1/model.safetensors.index.json
/kaggle/input/qwen2.5/transformers/14b-instruct-awq/1/model-00003-of-00003.safetensors
/kaggle/input/qwen2.5/transformers/14b-instruct-awq/1/config.json
/kaggle/input/qwen2.5/transformers/14b-instruct-awq/1/merges.txt
/kag

# CHECKING OUT MY GPU WORKINGW

*** repo: https://github.com/moonmehedi/Subchat-Trees-A-Scalable-Architecture-for-Multi-Threaded-Dialogue-and-Context-Isolation-in-LLM ***


In [2]:
! uv pip uninstall -q --system 'tensorflow'
! uv pip install -q --system 'vllm' 'triton==3.2.0' 'logits-processor-zoo' 'numpy<2'

In [3]:
import os
import re
import logging
from pathlib import Path
import pickle
import json
import joblib
import shutil
import glob
from tqdm.auto import tqdm
import warnings

import numpy as np
import pandas as pd



# For Qwen
import torch
import vllm
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor


INFO 12-18 17:51:31 [__init__.py:239] Automatically detected platform cuda.


In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GITHUB_TOKEN")
secret_value_1 = user_secrets.get_secret("GROQ_API_KEY")
secret_value_2 = user_secrets.get_secret("HuggingFACEHUB_access_token")
secret_value_3 = user_secrets.get_secret("LANGCHAIN_API_KEY")

# ✅ IMPORTANT: Set them in os.environ so other code can access them
os.environ["GITHUB_TOKEN"] = secret_value_0
os.environ["GROQ_API_KEY"] = secret_value_1
os.environ["HuggingFACEHUB_access_token"] = secret_value_2
os.environ["LANGCHAIN_API_KEY"] = secret_value_3
os.environ["LLM_BACKEND"] = "vllm"

# ✅ NEW: Set vLLM model path for backend config
# This will be used by config.py when backend starts
model_path = "/kaggle/input/qwen2.5/transformers/14b-instruct-awq/1"
os.environ["VLLM_MODEL_PATH"] = model_path

# Print the tokens (first 4 and last 4 characters for security)
print("="*60)
print("🔐 SECRETS LOADED AND SET IN ENVIRONMENT")
print("="*60)
print(f"✅ GITHUB_TOKEN: {secret_value_0[:4]}...{secret_value_0[-4:]}")
print(f"✅ GROQ_API_KEY: {secret_value_1[:4]}...{secret_value_1[-4:]}")
print(f"✅ HuggingFACEHUB_access_token: {secret_value_2[:4]}...{secret_value_2[-4:]}")
print(f"✅ LANGCHAIN_API_KEY: {secret_value_3[:4]}...{secret_value_3[-4:]}")
print(f"✅ LLM_BACKEND: vllm")
print(f"✅ VLLM_MODEL_PATH: {model_path}")
print("="*60)

🔐 SECRETS LOADED AND SET IN ENVIRONMENT
✅ GITHUB_TOKEN: gith...tWfg
✅ GROQ_API_KEY: gsk_...l6gr
✅ HuggingFACEHUB_access_token: hf_E...GaQC
✅ LANGCHAIN_API_KEY: lsv2...ea2f
✅ LLM_BACKEND: vllm
✅ VLLM_MODEL_PATH: /kaggle/input/qwen2.5/transformers/14b-instruct-awq/1


In [5]:
# vLLM V1 does not currently accept logits processor so we need to disable it
# https://docs.vllm.ai/en/latest/getting_started/v1_user_guide.html#deprecated-features
os.environ["VLLM_USE_V1"] = "0"

# Use 14B model (32B causes CUDA linker failures on Kaggle T4 GPUs)
#model_path = "/kaggle/input/qwen-3/transformers/14b-awq/1"
llm = vllm.LLM(
    model_path,
    quantization='awq',
    tensor_parallel_size=torch.cuda.device_count(),
    gpu_memory_utilization=0.91,
    trust_remote_code=True,
    dtype="half",
    enforce_eager=True,
    max_model_len=5120,
    disable_log_stats=True,
    enable_prefix_caching=True
)
tokenizer = llm.get_tokenizer()

INFO 12-18 17:51:59 [config.py:717] This model supports multiple tasks: {'generate', 'score', 'classify', 'embed', 'reward'}. Defaulting to 'generate'.
WARNING 12-18 17:52:00 [config.py:830] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 12-18 17:52:00 [config.py:830] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 12-18 17:52:00 [config.py:1770] Defaulting to use mp for distributed inference
WARNING 12-18 17:52:00 [cuda.py:93] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 12-18 17:52:00 [config.py:1770] Defaulting to use mp for distributed inference
WARNING 12-18 17:52:00 [cuda.py:93] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 12-18 17:52:00 [llm_engine.py:240] Initializing a V0 LLM engine (

[W1218 17:52:18.220254074 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W1218 17:52:18.221409606 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3


INFO 12-18 17:52:19 [utils.py:1055] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=951) INFO 12-18 17:52:19 [utils.py:1055] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=951) INFO 12-18 17:52:19 [pynccl.py:69] vLLM is using nccl==2.21.5
INFO 12-18 17:52:19 [pynccl.py:69] vLLM is using nccl==2.21.5
(VllmWorkerProcess pid=951) INFO 12-18 17:52:19 [pynccl.py:69] vLLM is using nccl==2.21.5
INFO 12-18 17:52:19 [pynccl.py:69] vLLM is using nccl==2.21.5


[W1218 17:52:19.500196904 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W1218 17:52:19.500909977 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3


INFO 12-18 17:52:19 [custom_all_reduce_utils.py:244] reading GPU P2P access cache from /root/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
(VllmWorkerProcess pid=951) INFO 12-18 17:52:19 [custom_all_reduce_utils.py:244] reading GPU P2P access cache from /root/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
INFO 12-18 17:52:19 [shm_broadcast.py:266] vLLM message queue communication handle: Handle(local_reader_ranks=[1], buffer_handle=(1, 4194304, 6, 'psm_58e15f04'), local_subscribe_addr='ipc:///tmp/557016ae-2c63-419d-91aa-73024d3b5971', remote_subscribe_addr=None, remote_addr_ipv6=False)
INFO 12-18 17:52:19 [parallel_state.py:1004] rank 0 in world size 2 is assigned as DP rank 0, PP rank 0, TP rank 0
(VllmWorkerProcess pid=951) INFO 12-18 17:52:19 [parallel_state.py:1004] rank 1 in world size 2 is assigned as DP rank 0, PP rank 0, TP rank 1
INFO 12-18 17:52:19 [shm_broadcast.py:266] vLLM message queue communication handle: Handle(local_reader_ranks=[1], buffer_handle=(1, 4194304, 6, 'psm_

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


(VllmWorkerProcess pid=951) INFO 12-18 17:52:29 [loader.py:458] Loading weights took 9.49 seconds
INFO 12-18 17:52:29 [loader.py:458] Loading weights took 9.53 seconds
(VllmWorkerProcess pid=951) INFO 12-18 17:52:29 [model_runner.py:1140] Model loading took 4.6720 GiB and 9.720133 seconds
INFO 12-18 17:52:29 [model_runner.py:1140] Model loading took 4.6720 GiB and 9.764329 seconds
(VllmWorkerProcess pid=951) INFO 12-18 17:52:29 [model_runner.py:1140] Model loading took 4.6720 GiB and 9.720133 seconds
INFO 12-18 17:52:29 [model_runner.py:1140] Model loading took 4.6720 GiB and 9.764329 seconds
(VllmWorkerProcess pid=951) INFO 12-18 17:52:36 [worker.py:287] Memory profiling takes 6.33 seconds
(VllmWorkerProcess pid=951) INFO 12-18 17:52:36 [worker.py:287] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.91) = 13.41GiB
(VllmWorkerProcess pid=951) INFO 12-18 17:52:36 [worker.py:287] model weights take 4.67GiB; non_torch_memory takes 0.12GiB; PyTorch

In [6]:
from vllm import SamplingParams

def stream_generate(llm, prompt):
    sampling_params = SamplingParams(
        temperature=0.2,
        top_p=0.9,
        max_tokens=512,
    )

    for output in llm.generate(
        [prompt],
        sampling_params,
        
    ):
        yield output.outputs[0].text


# ── Usage ────────────────────────────────────────────────
prompt = """You are a helpful assistant.
User: Explain tensor parallelism in simple terms.
Assistant:"""

for token in stream_generate(llm, prompt):
    print(token, end="", flush=True)


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

 Tensor parallelism is a technique used in deep learning and machine learning to speed up the training of large neural networks. Imagine you have a big puzzle, and you want to solve it faster. Instead of one person working on the whole puzzle, you can split the puzzle into smaller pieces and have multiple people work on different parts simultaneously. 

In the context of neural networks, a tensor is like a multi-dimensional array that holds data. Tensor parallelism means splitting these tensors into smaller parts and distributing them across multiple processors or machines. Each processor works on its part of the tensor, and then the results are combined to get the final output. This way, the overall computation is faster because multiple processors are working in parallel, rather than one processor doing all the work sequentially. This is particularly useful when dealing with very large models and datasets that require significant computational resources. By distributing the workload,

In [7]:
prompt = """what is quantum computing?"""

for token in stream_generate(llm, prompt):
    print(token, end="", flush=True)

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

 Quantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. In classical computing, data is processed using bits, which can represent either a 0 or a 1. In contrast, quantum computing uses quantum bits, or qubits, which can represent a 0, a 1, or both at the same time due to the principle of superposition. This allows quantum computers to perform certain types of calculations much faster than classical computers.

Here are some key concepts in quantum computing:

1. **Qubits**: The basic unit of quantum information, similar to bits in classical computing. Unlike classical bits, qubits can exist in a state of 0, 1, or any quantum superposition of these states.

2. **Superposition**: The principle that allows a qubit to be in multiple states simultaneously. This means a quantum computer can process a vast amount of possibilities simultaneously.

3. **Entanglement**: A quantum phenomenon where qub

In [8]:
print('h')

h


# test github

In [9]:
# Load secrets from Kaggle's secure environment
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

print("="*60)
print("🔐 LOADING SECRETS FROM KAGGLE")
print("="*60)

# Try to load GROQ_API_KEY
try:
    GROQ_API_KEY = user_secrets.get_secret("GROQ_API_KEY")
    os.environ["GROQ_API_KEY"] = GROQ_API_KEY
    print("✅ GROQ_API_KEY loaded successfully")
    print(f"   Key length: {len(GROQ_API_KEY)} characters")
except Exception as e:
    print(f"⚠️  GROQ_API_KEY not found: {e}")
    print("   Add it in Kaggle Settings → Secrets")
    GROQ_API_KEY = None

# Try to load GITHUB_TOKEN
try:
    GITHUB_TOKEN = user_secrets.get_secret("GITHUB_TOKEN")
    os.environ["GITHUB_TOKEN"] = GITHUB_TOKEN
    print("✅ GITHUB_TOKEN loaded successfully")
    print(f"   Token length: {len(GITHUB_TOKEN)} characters")
except Exception as e:
    print(f"⚠️  GITHUB_TOKEN not found: {e}")
    print("   Add it in Kaggle Settings → Secrets")
    GITHUB_TOKEN = None

# Set LLM backend to use vLLM (local model on Kaggle GPU)
os.environ["LLM_BACKEND"] = "vllm"  # We'll use the vLLM model loaded above
print("\n✅ LLM_BACKEND set to 'vllm' (using Kaggle GPU)")

print("="*60)

🔐 LOADING SECRETS FROM KAGGLE
✅ GROQ_API_KEY loaded successfully
   Key length: 56 characters
✅ GITHUB_TOKEN loaded successfully
   Token length: 93 characters

✅ LLM_BACKEND set to 'vllm' (using Kaggle GPU)


In [10]:
# Check GPU availability and configuration
import torch

print("="*60)
print("🔍 ENVIRONMENT CHECK")
print("="*60)
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
print(f"✅ CUDA version: {torch.version.cuda}")
print(f"✅ Number of GPUs: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"\n🎮 GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"   Memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.2f} GB")

print(f"\n✅ Current working directory: {os.getcwd()}")
print("="*60)

🔍 ENVIRONMENT CHECK
✅ PyTorch version: 2.6.0+cu124
✅ CUDA available: True
✅ CUDA version: 12.4
✅ Number of GPUs: 2

🎮 GPU 0: Tesla T4
   Memory: 14.74 GB

🎮 GPU 1: Tesla T4
   Memory: 14.74 GB

✅ Current working directory: /kaggle/working


In [11]:
# Clone the kaggle-run branch from GitHub (PUBLIC READ - no auth needed)
import subprocess

REPO_URL = "https://github.com/moonmehedi/Subchat-Trees-A-Scalable-Architecture-for-Multi-Threaded-Dialogue-and-Context-Isolation-in-LLM.git"
REPO_DIR = "Subchat-Trees"
BRANCH = "kaggle-run"

print("="*60)
print("📥 CLONING REPOSITORY")
print("="*60)

# Remove existing directory if present
if os.path.exists(REPO_DIR):
    print(f"⚠️  Removing existing {REPO_DIR} directory...")
    shutil.rmtree(REPO_DIR)

# Clone the specific branch (no authentication needed for public repos)
# Skip LFS files to avoid bandwidth quota issues
print(f"🔄 Cloning {BRANCH} branch (skipping LFS files)...")
print("   No authentication required for cloning (public repo)")

# Set environment variable to skip LFS files
clone_env = os.environ.copy()
clone_env["GIT_LFS_SKIP_SMUDGE"] = "1"

result = subprocess.run(
    ["git", "clone", "-b", BRANCH, "--single-branch", REPO_URL, REPO_DIR],
    capture_output=True,
    text=True,
    env=clone_env
)

if result.returncode == 0:
    print(f"✅ Successfully cloned {BRANCH} branch!")
    print(f"📂 Repository location: {os.path.abspath(REPO_DIR)}")
    
    # Pull Git LFS files for scenarios only (saves bandwidth)
    print("\n📥 Pulling Git LFS scenario files...")
    os.chdir(REPO_DIR)
    lfs_result = subprocess.run(
        ["git", "lfs", "pull", "--include=backend/dataset/scenarios/*.json"],
        capture_output=True,
        text=True
    )
    
    if lfs_result.returncode == 0:
        print("✅ Successfully pulled scenario files from Git LFS")
    else:
        print(f"⚠️  Warning: Git LFS pull returned code {lfs_result.returncode}")
        if lfs_result.stderr:
            print(f"   {lfs_result.stderr}")
    
    os.chdir("..")  # Return to parent directory
    
    # List key directories to verify
    print("\n📁 Key directories found:")
    key_dirs = ["backend", "backend/src", "backend/dataset"]
    for dir_path in key_dirs:
        full_path = os.path.join(REPO_DIR, dir_path)
        if os.path.exists(full_path):
            print(f"   ✅ {dir_path}")
        else:
            print(f"   ❌ {dir_path} (not found)")
else:
    print(f"❌ Clone failed: {result.stderr}")
    
print("="*60)

📥 CLONING REPOSITORY
⚠️  Removing existing Subchat-Trees directory...
🔄 Cloning kaggle-run branch (skipping LFS files)...
   No authentication required for cloning (public repo)
✅ Successfully cloned kaggle-run branch!
📂 Repository location: /kaggle/working/Subchat-Trees

📥 Pulling Git LFS scenario files...
✅ Successfully pulled scenario files from Git LFS

📁 Key directories found:
   ✅ backend
   ✅ backend/src
   ✅ backend/dataset


In [12]:
# Configure git identity
os.chdir(REPO_DIR)

print("="*60)
print("⚙️  CONFIGURING GIT")
print("="*60)

!git config user.name "moonmehedi"
!git config user.email "the.mehedi.hasan.moon@gmail.com"

print("✅ Git identity configured!")
print(f"   User: moonmehedi")
print(f"   Email: the.mehedi.hasan.moon@gmail.com")

# Verify current branch
branch_result = subprocess.run(["git", "branch", "--show-current"], capture_output=True, text=True)
print(f"\n✅ Current branch: {branch_result.stdout.strip()}")
print("="*60)

os.chdir("..")  # Return to parent directory

⚙️  CONFIGURING GIT


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


✅ Git identity configured!
   User: moonmehedi
   Email: the.mehedi.hasan.moon@gmail.com

✅ Current branch: kaggle-run


In [13]:
def create_test_log(log_dir="kaggle_logs", log_file="connection_test.log"):
    """Create a detailed test log with GPU and environment info"""
    from datetime import datetime
    
    os.makedirs(log_dir, exist_ok=True)
    log_path = os.path.join(log_dir, log_file)
    current_time = datetime.now()
    
    with open(log_path, "w") as f:
        f.write("="*60 + "\n")
        f.write("🔬 KAGGLE GPU TEST RUN - CONNECTION VERIFICATION\n")
        f.write("="*60 + "\n\n")
        
        f.write(f"📅 Test Date: {current_time.strftime('%Y-%m-%d')}\n")
        f.write(f"⏰ Test Time: {current_time.strftime('%H:%M:%S UTC')}\n")
        f.write(f"📍 Timestamp: {pd.Timestamp.now()}\n\n")
        
        f.write("="*60 + "\n")
        f.write("🎮 GPU CONFIGURATION\n")
        f.write("="*60 + "\n")
        f.write(f"GPU Count: {torch.cuda.device_count()}\n")
        
        if torch.cuda.is_available():
            for i in range(torch.cuda.device_count()):
                f.write(f"\nGPU {i}:\n")
                f.write(f"  - Name: {torch.cuda.get_device_name(i)}\n")
                f.write(f"  - Memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.2f} GB\n")
        else:
            f.write("⚠️  No GPU detected\n")
        
        f.write("\n" + "="*60 + "\n")
        f.write("📊 ENVIRONMENT INFO\n")
        f.write("="*60 + "\n")
        f.write(f"PyTorch Version: {torch.__version__}\n")
        f.write(f"CUDA Available: {torch.cuda.is_available()}\n")
        f.write(f"Working Directory: {os.getcwd()}\n")
        
        f.write("\n" + "="*60 + "\n")
        f.write("✅ TEST STATUS: SUCCESS\n")
        f.write("="*60 + "\n")
        f.write(f"\nThis log was generated from Kaggle notebook\n")
        f.write(f"Push attempt at: {current_time.isoformat()}\n")
    
    return log_path, current_time


def git_commit_and_push(file_path, commit_message, branch="kaggle-run"):
    """Commit a file and push to GitHub"""
    import subprocess
    
    # Add file
    add_result = subprocess.run(["git", "add", file_path], capture_output=True, text=True)
    if add_result.returncode != 0:
        return False, f"Git add failed: {add_result.stderr}"
    
    # Commit
    commit_result = subprocess.run(["git", "commit", "-m", commit_message], capture_output=True, text=True)
    if commit_result.returncode != 0:
        return False, f"Git commit failed: {commit_result.stderr}"
    
    # Push with token
    if "GITHUB_TOKEN" not in os.environ:
        return False, "GITHUB_TOKEN not found in environment"
    
    repo_url_with_token = f"https://{os.environ['GITHUB_TOKEN']}@github.com/moonmehedi/Subchat-Trees-A-Scalable-Architecture-for-Multi-Threaded-Dialogue-and-Context-Isolation-in-LLM.git"
    
    # Set remote URL
    subprocess.run(["git", "remote", "set-url", "origin", repo_url_with_token], capture_output=True)
    
    # Push
    push_result = subprocess.run(["git", "push", "origin", branch], capture_output=True, text=True)
    
    if push_result.returncode == 0:
        return True, "Push successful"
    else:
        return False, f"Push failed: {push_result.stderr}"


# Main execution
print("="*60)
print("🧪 TESTING GIT PUSH CAPABILITY")
print("="*60)

try:
    # Change to repo directory
    os.chdir(REPO_DIR)
    
    # Create test log
    log_path, timestamp = create_test_log()
    print(f"✅ Created detailed test log: {log_path}")
    
    # Commit and push
    commit_msg = f"Test: Kaggle GPU verification - {timestamp.strftime('%Y-%m-%d %H:%M:%S')}"
    success, message = git_commit_and_push(log_path, commit_msg, BRANCH)
    
    if success:
        print("\n✅ Successfully pushed to GitHub!")
        print(f"   📁 Check: {log_path}")
        print(f"   📅 Pushed at: {timestamp.strftime('%Y-%m-%d %H:%M:%S')}")
        print("   💡 Pull on your local machine to verify sync")
    else:
        print(f"\n❌ {message}")
        
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()
finally:
    # Always return to parent directory
    os.chdir("..")

print("="*60)

🧪 TESTING GIT PUSH CAPABILITY
✅ Created detailed test log: kaggle_logs/connection_test.log

✅ Successfully pushed to GitHub!
   📁 Check: kaggle_logs/connection_test.log
   📅 Pushed at: 2025-12-18 17:53:20
   💡 Pull on your local machine to verify sync


# 🔗 Step 5: Integrate vLLM with Backend

**This connects the loaded vLLM model to your backend code**

In [14]:
# Register the vLLM model with the backend
import sys
sys.path.insert(0, os.path.join(REPO_DIR, "backend"))

from src.services.vllm_client import VLLMClient

print("="*60)
print("🔗 INTEGRATING vLLM WITH BACKEND")
print("="*60)

# Register the globally loaded vLLM model
VLLMClient.set_model(llm)

print("✅ vLLM model is now available to backend services")
print(f"   Model: {model_path}")
print(f"   GPUs: {torch.cuda.device_count()}")
print(f"   Backend will use LLM_BACKEND={os.getenv('LLM_BACKEND')}")
print("="*60)

🔗 INTEGRATING vLLM WITH BACKEND
✅ vLLM model registered: /kaggle/input/qwen2.5/transformers/14b-instruct-awq/1
✅ vLLM model is now available to backend services
   Model: /kaggle/input/qwen2.5/transformers/14b-instruct-awq/1
   GPUs: 2
   Backend will use LLM_BACKEND=vllm


In [15]:
pip install -r /kaggle/working/Subchat-Trees/backend/requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# 🧪 Step 6: Test vLLM Integration

**Quick test to verify backend can use vLLM on Kaggle GPU**

In [16]:
# Test the backend with vLLM
from src.services.simple_llm import SimpleLLMClient
from src.models.tree import TreeNode

print("="*60)
print("🧪 TESTING BACKEND WITH vLLM")
print("="*60)

# Create a simple test
llm_client = SimpleLLMClient()

# Create a test node (TreeNode creates its own buffer internally)
# Note: TreeNode uses 'node_id' (not 'id'), and buffer_size (not buffer object)
root = TreeNode(
    node_id="test", 
    title="Test Conversation", 
    buffer_size=5,
    llm_client=llm_client
)

# Add a message to the node's buffer
root.buffer.add_message("user", "Hello, test message")

# Test generation
print("\n📝 Testing response generation...")
response = llm_client.generate_response(root, "What is 2+2?")

print(f"\n✅ Response: {response}")
print(f"📊 Token usage: {llm_client.get_last_usage()}")
print("\n" + "="*60)
print("✅ Backend integration successful!")
print("="*60)

✅ Using vLLM backend with Kaggle GPU: /kaggle/input/qwen2.5/transformers/14b-instruct-awq/1
🧪 TESTING BACKEND WITH vLLM
✅ vLLM connected for RESPONSES: /kaggle/input/qwen2.5/transformers/14b-instruct-awq/1
✅ vLLM will be used for SUMMARIZATION: /kaggle/input/qwen2.5/transformers/14b-instruct-awq/1
📊 Buffer size: 5 messages | Summarization will trigger every 5 messages
📋 Buffer (1/5): Last 3 messages (full log in file)
   1. [user] Hello, test message

📝 Testing response generation...
*******************context*********************
 [{'role': 'user', 'content': 'Hello, test message'}]


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


✅ Response: Hello! It looks like you might be testing or starting a conversation. How can I assist you today? If you have any specific questions or need help with something, feel free to ask!
📊 Token usage: {'prompt_tokens': 23, 'completion_tokens': 42, 'total_tokens': 112}

✅ Backend integration successful!


# testing ends

# 🚀 Step 7: Start Backend Server

**Run the FastAPI server with vLLM backend on Kaggle GPU**

In [17]:
print("🚀 SETUP COMPLETE - READY FOR SUBCHAT TREES EXECUTION")

🚀 SETUP COMPLETE - READY FOR SUBCHAT TREES EXECUTION


In [18]:
pwd

'/kaggle/working'

In [ ]:
# ⚠️ IMPORTANT: vLLM model must be registered in the SAME process as the server
# We'll use nest_asyncio to allow uvicorn to run in Jupyter's existing event loop

import uvicorn
import os
import nest_asyncio

# Allow nested event loops (required for Jupyter)
nest_asyncio.apply()

# Ensure backend is in path
import sys
sys.path.insert(0, f"/kaggle/working/{REPO_DIR}/backend")

# Re-register vLLM model (in case it was lost)
from src.services.vllm_client import VLLMClient
VLLMClient.set_model(llm)

print("="*60)
print("🚀 STARTING BACKEND SERVER IN NOTEBOOK KERNEL")
print("="*60)
print(f"📂 Backend path: /kaggle/working/{REPO_DIR}/backend")
print(f"🔧 Backend mode: {os.getenv('LLM_BACKEND')}")
print(f"✅ vLLM model registered: {VLLMClient.is_available()}")
print(f"🎯 Server URL: http://0.0.0.0:8000")
print("="*60)
print("\n⚠️  Server starting... (will block this cell)")
print("💡 Stop with: Kernel → Interrupt\n")

# Change to backend directory
os.chdir(f"/kaggle/working/{REPO_DIR}/backend")

# Start the server programmatically (same process, can access llm variable)
# nest_asyncio allows this to work in Jupyter's existing event loop
uvicorn.run("src.main:app", host="0.0.0.0", port=8000, reload=False)

✅ vLLM model registered: /kaggle/input/qwen2.5/transformers/14b-instruct-awq/1
🚀 STARTING BACKEND SERVER IN NOTEBOOK KERNEL
📂 Backend path: /kaggle/working/Subchat-Trees/backend
🔧 Backend mode: vllm
✅ vLLM model registered: True
🎯 Server URL: http://0.0.0.0:8000

⚠️  Server starting... (will block this cell)
💡 Stop with: Kernel → Interrupt

✅ vLLM connected for RESPONSES: /kaggle/input/qwen2.5/transformers/14b-instruct-awq/1
✅ vLLM will be used for SUMMARIZATION: /kaggle/input/qwen2.5/transformers/14b-instruct-awq/1


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

✅ Created fresh vector collection with all-mpnet-base-v2 embeddings (0 messages)
✅ Initialized multi-query decomposition + context windows
✅ Vector index enabled for RAG
✅ All logs cleared on server startup


INFO:     Shutting down


(VllmWorkerProcess pid=951) INFO 12-18 18:34:33 [multiproc_worker_utils.py:259] Worker exiting


RuntimeError: Event loop stopped before Future completed.

INFO 12-18 18:34:34 [multiproc_worker_utils.py:124] Killing local vLLM worker processes
